In [0]:
df = spark.table('social_media_clean')
df.createTempView('df')

## Identify the most popular action by count.

In [0]:
%sql
SELECT FIRST(action) AS most_popular_action
  FROM (
    SELECT action, COUNT(*) as count
      FROM df
      GROUP BY action
      ORDER BY count
  )

## Identify the most popular device type for particular action's title

In [0]:
%sql
WITH tmp (
  SELECT action, device, COUNT(*) as count
  FROM df
  GROUP BY action, device
  ORDER BY action, count DESC
)

SELECT action, device, count
FROM (
  SELECT *, ROW_NUMBER() OVER (
    PARTITION BY action
    ORDER BY count DESC
  ) AS rw
  FROM tmp
)
WHERE rw = 1
ORDER BY count DESC

## Identify the most active users by actions count

In [0]:
%sql
WITH tmp AS (
  SELECT ip, action, COUNT(*) AS count
  FROM df
  GROUP BY action, ip
  ORDER BY count DESC
)

SELECT FIRST(action) as action, collect_list(ip) as users, FIRST(count) as count
FROM (
  SELECT action, ip, count, RANK() OVER (
    PARTITION BY action
    ORDER BY count DESC
  ) AS rw
  FROM tmp
)
WHERE rw = 1
GROUP BY action, count
ORDER BY count DESC

## Identify the most popular devices type which was used for "dislikes"

In [0]:
%sql
SELECT FIRST(device)
FROM (
  SELECT device, COUNT(*)
  FROM df
  WHERE action = 'dislike'
  GROUP BY device
  ORDER BY COUNT(*) DESC
)